In [1]:
from tqdm import tqdm
import pandas as pd
from datasets import Dataset
from src.dataset import SPFastaDatasetBinary, SPFastaDatasetBinaryWithTokenizedCategory
import torch
from torch import nn
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert", device="cpu")

/home/andrebw/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = SPFastaDatasetBinaryWithTokenizedCategory("data/train.fasta")
dataset_train = Dataset.from_pandas(ds.data).with_format("torch", device=device)
del ds

  0%|          | 0/20290 [00:00<?, ?it/s]

100%|██████████| 20290/20290 [00:00<00:00, 542368.05it/s]


In [3]:
# Tokenize:
dataset_train = dataset_train.map(lambda x: tokenizer(x['sequence'], return_tensors="pt", padding=True), batched=True)
dataset_train

Map: 100%|██████████| 20290/20290 [00:01<00:00, 12991.73 examples/s]


Dataset({
    features: ['sequence', 'label', 'uniprot_ac', 'kingdom', 'type', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 20290
})

In [4]:
dataloader = torch.utils.data.DataLoader(dataset_train, batch_size=2, shuffle=True)

# Define model

In [11]:
from transformers import AutoModelForSequenceClassification

class ProtBertSequenceClassification:
    def __init__(self, device: str=device) -> None:
        self.device = device

        self.model = AutoModelForSequenceClassification.from_pretrained("Rostlab/prot_bert", num_labels=2).to(device)
        self.model.train()
        self.model.label2id = {
            'NO_SP': 0,
            'SP': 1
        }
        self.model.id2label= {
            0: 'NO_SP',
            1: 'SP'
        }
        self.tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert", device=device)

    def forward(self, x):
        logits = self.model.forward(x).logits
        return nn.functional.softmax(logits, dim=-1)
    
    def loss(self, x, y):
        pred = self.forward(x)
        


In [12]:
sample = dataset_train[0]
model = ProtBertSequenceClassification()
# model.forward(input_ids=sample['input_ids'].reshape(1, -1), attention_mask=sample['attention_mask'].reshape(1, -1)).logits

: 